# Quantiles

https://machinelearningmastery.com/curve-fitting-with-python/

https://study.com/academy/lesson/interpolation-in-statistics-definition-formula-example.html

In [1]:
import time
import pandas as pd

import numpy as np

from numpy import corrcoef as pcor

from numpy import exp as exp
from scipy import spatial

from itertools import combinations

import pickle

from datetime import datetime, timedelta
import random
import glob


import warnings
warnings.filterwarnings("ignore")
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import numpy as np
def return_best_shapelet_pearson(vector):
    correlation_lst = []
    corrs = []
    for i in range(len(shapelet_standard_array)):
        score = similarity_metrix(shapelet_standard_array[i],vector)
        correlation_lst.append(shapelet_standard_names[i])
        corrs.append(score)
    scenario = corrs.index(max(corrs))
    return correlation_lst[scenario]

    
def return_all_shapelet_pearson(vector):
#     correlation_lst = []
    corrs = []
    for i in range(len(shapelet_standard_array)):
        score = similarity_metrix(shapelet_standard_array[i],vector)
#         correlation_lst.append(shapelet_standard_names[i])
        corrs.append(score)
    return corrs



def similarity_metrix(vector1,vector2):
    '''
    Here we have given user the flexibility to change the similarity function. Currently we have made it pearson correlation but it can be cosine
    
    1 - spatial.distance.cosine(vector1, vector2)
    '''
    similarity_value = round(pcor(vector1,vector2)[0][1],3)
    return similarity_value



In [43]:
export_visualizations = False ### when this is set to True, visualizations will be expored. If False, visualizations will only be displayed not exported
Data_refresh = 0 

# Smoothening_param= 3 ### make it dynamic in code below 


### if it is set to 1, data processing of all models needs to be done. As is this needs to be set1 when we feel model data need to be updated else 0


#### Ignore model List 
Ignore_model_list  = []

Ignore_State_list = ['Illinois', 'Arizona', 'Massachusetts',
       'Wisconsin', 'Texas', 'Nebraska', 'Utah', 'Oregon','United States','Washington',
       'New York', 'Rhode Island', 'Georgia', 'New Hampshire',
       'North Carolina', 'New Jersey', 'Colorado', 'Maryland', 'Nevada',
       'Tennessee', 'Hawaii', 'Indiana', 'Kentucky', 'Minnesota',
       'Oklahoma', 'Pennsylvania', 'South Carolina',
       'District of Columbia', 'Kansas', 'Missouri', 'Vermont',
       'Virginia', 'Connecticut', 'Iowa', 'Louisiana', 'Ohio', 'Michigan',
       'South Dakota', 'Arkansas', 'Delaware', 'Mississippi',
       'New Mexico', 'North Dakota', 'Wyoming', 'Alaska', 'Maine',
       'Alabama', 'Idaho', 'Montana', 'Puerto Rico', 'Virgin Islands',
       'Guam', 'West Virginia', 'Northern Mariana Islands',
       'American Samoa']

# select_state_list = ["Florida"]

# State_list = ["Florida"]


vector_length = (0,4)   ### 1 means using N-1 week value for defining shapelet and 4 weeks 4 weeks from future. 4 can't be changed because models generate only 4 weeks ahead predictions.
## (0,4) means look 4 weeks ahead in future while defining shapelet
## (1,4) means look 4 weeks ahead in future, 1 week in past basically N-1 week from actual covid incidence list for defining shapelet

history_weeks = vector_length[0]

future_weeks = vector_length[1]

assert future_weeks<=4,"Looking 4 weeks in future is fixed because our modelsgenerate 4 weeks ahead predictions. \n Please change vector_length[1]"


Number_of_shapelets = 6 ### 6 

global Shapelet_length
Shapelet_length = vector_length[0]+vector_length[1]

shapelet_standard_array = [[0]*Shapelet_length for w in range(Number_of_shapelets)]
## here we have initialized an zero valued array of array.


# Hyper Params 

shapelet_standard_names = ["Flat","Inc",'Dec',"Surge",'Peaking',"Near Peak"]

assert len(shapelet_standard_names)==Number_of_shapelets, 'Size of array mismatch for shapelet_standard_names and value of  Number_of_shapelets'
# print('Size Mismacth')

### for this experiment, we have defined following shapelets 
# flat = [1.0, 1.00000001, 1.00000002, 1.00000003, 1.00000004]
# stable_inc = [1 ,2, 3, 4, 5]#, i.e., linear
# stable_dec = [5, 4, 3 ,2, 1]#, i.e., linear but decreasing
# surge =[exp(-1/2), exp(0/2), exp(1/2), exp(2/2), exp(3/2)]#, i.e., like exp(x)
# peaking = [-1*w for w in [exp(1/2) ,exp(0/2), exp(-1/2), exp(-2/2), exp(-3/2)]]#, i.e., like -exp(-x)
# at_near_peak=  [-1*w for w in [exp(-1/2) ,exp(0/2) ,exp(1/2) ,exp(2/2), exp(3/2)]]#, i.e., like -exp(x)

shapelet_standard_array[0] = [1.0, 0, 1.0, 0.0000]
shapelet_standard_array[1] = [1 ,2, 3, 4]
shapelet_standard_array[2] = [5, 4, 3 ,2]
shapelet_standard_array[3] = [1,2,4,8]
shapelet_standard_array[4] = [-1*w for w in [exp(1/2) ,exp(0/2), exp(-1/2), exp(-2/2)]]
shapelet_standard_array[5] = [-1*w for w in [exp(-1/2) ,exp(0/2) ,exp(1/2) ,exp(2/2)]]

assert len(shapelet_standard_array[0])==Shapelet_length, 'Size of defined shapelet array mismatch for shapelet_standard_names and value of  Shapelet_length.please check vector_length'



In [80]:

def valid_week_nbr(a_string):

    if [int(word) for word in a_string.split() if word.isdigit()][0]<5:
        return 1
    else:
        return 0
def interpolate(test_array):
    inter_list= []
    array_x = [w[0] for w in test_array]
    array_y = [w[1] for w in test_array]
#     sum_iter = 0
    for i in range(50):
        value_x = np.random.uniform(low=0.0, high=1.0)
        y_inter = np.interp(value_x,array_x,array_y)
        inter_list.append(round(y_inter,2))
    return inter_list
def return_Similarity_score_all(x):
    try:
    #     print(x[0][0])
    #     frst_week = []
        for item in x:
            if item[0].find("1")>-1:
                frst_week=item[1]
                break
        second_week = []
        for item in x:
            if item[0].find("2")>-1:
                second_week=item[1]
                break
        thrd_week =[]
        for item in x:
            if item[0].find("3")>-1:
                thrd_week=item[1]
                break
        frth_week=[]
        for item in x:
            if item[0].find("4")>-1:
                frth_week=item[1]
                break
        interpolated_array = []


        for i in range(len(frst_week)):
            vector = [frst_week[i],second_week[i],thrd_week[i],frth_week[i]]
    #         print(vector)
            interpolated_array.append(return_all_shapelet_pearson(vector))


        return interpolated_array
    except:
        return "NAN"
    
def aggregated_similarity(data):
    try:
        a = np.array(data)

        aggs = a.mean(axis=0)

        xx = [round(w,4) for w in aggs]
        return xx
    except:
        print(data)
    
from datetime import date


d0 = date(2020, 1, 22)

def format_dt_int(w):
    wsplit = w.split("-")
    yr = int(wsplit[0])
    mth=int(wsplit[1])
    dt = int(wsplit[2])
    d1 = date(yr, mth, dt)
    delta = d1 - d0
    return delta.days

import math
def is_valid(x):
    for i in x:
        if math.isnan(i):
            return 0
    return 1             


In [4]:
Source_path = "../Data_Sources/quantile_preprocessing/"

In [5]:
locatoins = pd.read_csv(Source_path+"locations.csv")

In [6]:
location_mapping = locatoins[['location','location_name']].set_index("location").to_dict()['location_name']

In [59]:
location_mapping['US']='United States'

In [60]:
model_name = 'CEID-Walk'

runtype = "case"

valid_vals = ['US',	 '01',	 '02',	 '04',	 '05',	 '06',	 '08',	 '09',	 '10',	 '11',	 '12',	 '13',	 '15',	 '16',	 '17',	 '18',	 '19',	 '20',	 '21',	 '22',	 '23',	 '24',	 '25',	 '26',	 '27',	 '28',	 '29',	 '30',	 '31',	 '32',	 '33',	 '34',	 '35',	 '36',	 '37',	 '38',	 '39',	 '40',	 '41',	 '42',	 '44',	 '45',	 '46',	 '47',	 '48',	 '49',	 '50',	 '51',	 '53',	 '54',	 '55',	 '56',	 '60',	 '66',	 '69',	 '72',	 '74',	 '78']


In [8]:
## Load source files

In [9]:
#path = Source_path+"2020-08-09-CEID-Walk.csv"

In [10]:
#test_file = pd.read_csv(Source_path+"2020-08-09-CEID-Walk.csv")

In [87]:
# path = "/Users/satwant/Downloads/CEID-Walk/"

root = '/Users/satwant/Downloads/covid19-forecast-hub-master/data-processed'

In [89]:
model_root = glob.glob(root+"/*")

In [12]:
model_master_df = pd.DataFrame()
for path in model_root:
    filenames = glob.glob(path+"2*.csv") 

In [61]:
    master_data = pd.DataFrame()
    for i in filenames:
        tmp = pd.read_csv(i)
        ## filter for runtype
        cases_data_tmp = tmp[tmp['target'].str.contains(runtype)]

        ## filter for quantiles only 

        cases_data_tmp1 = cases_data_tmp[cases_data_tmp['type']=='quantile']

        cases_data = cases_data_tmp1[cases_data_tmp1['location'].isin(valid_vals)]

        cases_data['location'] = cases_data['location'].apply(lambda x : location_mapping.get(x,x)) 

        cases_data = cases_data[~cases_data['location'].isin(Ignore_State_list)]

        cases_data['valid_row'] = cases_data['target'].apply(valid_week_nbr)

        cases_data = cases_data[cases_data['valid_row']==1]

        master_data = pd.concat([master_data,cases_data],axis=0)
    
    

In [64]:
cases_data = master_data.copy()

In [65]:
cases_data['ordered_pair'] =cases_data[['quantile', 'value']].apply(tuple, axis=1)



In [66]:
cases_subset = cases_data[['forecast_date','location','target','ordered_pair','target_end_date']]


cases_rolledup = cases_subset.groupby(['forecast_date','location','target','target_end_date'])['ordered_pair'].apply(list).reset_index()



In [67]:
cases_rolledup['IR_value_50sim'] = cases_rolledup['ordered_pair'].apply(interpolate)

In [68]:
cases_rolledup['week_interpolated_array'] =cases_rolledup[['target', 'IR_value_50sim']].apply(tuple, axis=1)


In [69]:
cases_rolledup1 = cases_rolledup.groupby(['forecast_date','location'])['week_interpolated_array'].apply(list).reset_index()


In [71]:

cases_rolledup1['All_iterpol_similar'] = cases_rolledup1['week_interpolated_array'].apply(return_Similarity_score_all)


In [73]:
cases_rolledup1 = cases_rolledup1[cases_rolledup1['All_iterpol_similar']!='NAN']

In [74]:

cases_rolledup1['aggregated_similarity'] = cases_rolledup1['All_iterpol_similar'].apply(aggregated_similarity)


In [75]:
cases_rolledup1['forecast_date1'] = cases_rolledup1['forecast_date'].apply(format_dt_int)


In [76]:
cases_rolledup1['model_name'] = model_name

In [84]:
cases_rolledup1_subset = cases_rolledup1[['forecast_date','model_name','forecast_date1','location','aggregated_similarity']]

In [85]:
cases_rolledup1_subset['valid_row'] = cases_rolledup1_subset['aggregated_similarity'].apply(is_valid)

cases_rolledup1_subset = cases_rolledup1_subset[cases_rolledup1_subset["valid_row"]==1]

In [86]:
cases_rolledup1_subset

,forecast_date,model_name,forecast_date1,location,aggregated_similarity,valid_row
0,2020-08-09,CEID-Walk,200,California,"[0.1535, -0.1034, 0.1034, -0.1334, -0.0818, 0....",1
1,2020-08-09,CEID-Walk,200,Florida,"[0.0295, -0.0264, 0.0264, -0.0574, -0.0026, 0....",1
2,2020-08-16,CEID-Walk,207,California,"[0.1997, -0.1374, 0.1374, -0.1902, -0.0985, 0....",1
3,2020-08-16,CEID-Walk,207,Florida,"[-0.0302, -0.0335, 0.0335, -0.034, -0.0287, 0....",1
4,2020-08-23,CEID-Walk,214,California,"[-0.1841, 0.0682, -0.0682, 0.119, 0.0357, -0.1...",1
...,...,...,...,...,...,...
127,2022-04-25,CEID-Walk,824,Florida,"[-0.0136, -0.0014, 0.0014, 0.0228, -0.0186, -0...",1
128,2022-05-02,CEID-Walk,831,California,"[0.061, 0.0158, -0.0158, 0.012, 0.0137, -0.0136]",1
129,2022-05-02,CEID-Walk,831,Florida,"[0.1524, 0.0962, -0.0962, 0.0511, 0.1132, -0.0...",1
130,2022-05-09,CEID-Walk,838,California,"[0.0942, 0.1059, -0.1059, 0.1154, 0.0843, -0.1...",1
